# Aprendizaje Supervisado de un Conjunto de Datos del Clima

En este notebook se muestra un benchmark de distintos metodos de aprendizaje supervisado, en este caso es un conjunto de datos climatico en Australia.

Este set de datos puede ser descargado del siguiente link:

https://www.kaggle.com/jsphyg/weather-dataset-rattle-package

Se iniciara por cargar las bibliotecas necesarias.

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import preprocessing
%matplotlib inline

## Preprocesamiento

En esta seccion se realiza el pre-procesamiento sobre el conjunto de datos, este proviente de la investigacion anterior y por lo tanto no es explicara el racionamiento de este pre-procesamiento

In [2]:
weather_results = pd.read_csv("weatherAUS.csv")
weather_results.replace(to_replace='Yes', value = 1, inplace = True)
weather_results.replace(to_replace='No',  value = 0, inplace = True)

directions = ["N","NNE","NE","ENE","E","ESE", "SE", "SSE","S","SSW","SW","WSW","W","WNW","NW","NNW"]

def compassToDeg(compass_direction):
    global directions
    index = directions.index(compass_direction)
    angle = index * 22.5
    return angle

def windGustDirConvert(direction):
    global directions
    if direction['WindGustDir'] in directions:
        return compassToDeg(direction['WindGustDir'])

def windDir9amConvert(direction):
    global directions
    if direction['WindDir9am'] in directions:
        return compassToDeg(direction['WindDir9am'])

def windDir3pmConvert(direction):
    global directions
    if direction['WindDir3pm'] in directions:
        return compassToDeg(direction['WindDir3pm'])

windGustDirAngle = weather_results.filter(regex=r'WindGustDir').apply(windGustDirConvert, axis=1)
windDir9amAngle = weather_results.filter(regex=r'WindDir9am').apply(windDir9amConvert, axis=1)
windDir3pmAngle = weather_results.filter(regex=r'WindDir3pm').apply(windDir3pmConvert, axis=1)

windGustDirAngleCos = windGustDirAngle.apply(np.cos)
windGustDirAngleSin = windGustDirAngle.apply(np.sin)
windDir9amAngleCos = windDir9amAngle.apply(np.cos)
windDir9amAngleSin = windDir9amAngle.apply(np.sin)
windDir3pmAngleCos = windDir3pmAngle.apply(np.cos)
windDir3pmAngleSin = windDir3pmAngle.apply(np.sin)

del weather_results['WindGustDir']
del weather_results['WindDir9am']
del weather_results['WindDir3pm']

weather_results['windGustDirAngleCos'] = windGustDirAngleCos
weather_results['windGustDirAngleSin'] = windGustDirAngleSin
weather_results['windDir9amAngleCos'] = windDir9amAngleCos
weather_results['windDir9amAngleSin'] = windDir9amAngleSin
weather_results['windDir3pmAngleCos'] = windDir3pmAngleCos
weather_results['windDir3pmAngleSin'] = windDir3pmAngleSin

weather_results.fillna(weather_results.mean(), inplace = True)

## Metodos Supervisados

% Class Methods
### Regresion Linear? (Clasificacion) o SVM (Regresion)
### kNN
### Naive Bayes
### Regresion Decision Tress
### Random Forest 
### Kernel SVM


% New Methods
### Stochastic Gradient Descent?
### Gaussian Process Regression/Classification
### Cross Decomposition?
### AdaBoost
### Forest of randomized trees
### Isotonic Regression

## Comparacion de Resultados